# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weathercsv_df = pd.read_csv('../WeatherPy/output_data/weather_data.csv', index_col=0)

weathercsv_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Ixtapa,MX,20.7000,-105.2000,59.00,94,40,3.44,1612795091
1,Bokovskaya,RU,49.2290,41.8306,18.97,82,100,7.61,1612795091
2,Severo-Kuril'sk,RU,50.6789,156.1250,18.68,93,87,6.20,1612795091
3,Codrington,AU,-38.2667,141.9667,54.36,87,23,9.66,1612795091
4,Thompson,CA,55.7435,-97.8558,-25.60,68,90,14.97,1612795091
...,...,...,...,...,...,...,...,...,...
530,Ibrā’,OM,22.6906,58.5334,70.30,41,100,8.23,1612795091
531,Beisfjord,NO,68.3751,17.5988,30.20,86,40,3.44,1612795091
532,Severodvinsk,RU,64.5635,39.8302,-13.00,77,20,2.24,1612795091
533,Turukhansk,RU,65.8167,87.9833,4.82,94,100,7.94,1612795091


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

sample_df = weathercsv_df.head(10)
locations = weathercsv_df[["Lat", "Lng"]].astype(float)
humidity = weathercsv_df['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
